Tahap 3: Feature Engineering & Preprocessing
1. Objectives
Feature Creation: Membuat rasio keuangan baru yang lebih relevan bagi model.

Handling Outliers: Mengatasi nilai ekstrem agar tidak merusak distribusi data.

Encoding: Mengubah data kategorikal menjadi numerik.

Scaling: Menstandarisasi rentang nilai agar model stabil.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

# Load data hasil EDA
df = pd.read_csv("../data/processed/personal_finance_cleaned.csv")

# Pastikan record_date dalam format datetime
df['record_date'] = pd.to_datetime(df['record_date'])

In [2]:
# 1. Monthly Disposable Income (Sisa uang setelah kebutuhan pokok dan cicilan)
df['disposable_income'] = df['monthly_income_usd'] - df['monthly_expenses_usd'] - df['monthly_emi_usd']

# 2. Expense-to-Income Ratio (Seberapa besar porsi gaji untuk hidup)
df['expense_to_income_ratio'] = df['monthly_expenses_usd'] / df['monthly_income_usd']

# 3. Loan-to-Income Ratio
df['loan_to_income_ratio'] = df['loan_amount_usd'] / (df['monthly_income_usd'] * 12)

# 4. Age Groups (Mengelompokkan umur untuk menangkap pola perilaku generasi)
bins = [18, 25, 35, 50, 70]
labels = ['Gen Z', 'Millennials', 'Gen X', 'Boomers']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

print("Fitur baru berhasil dibuat.")

Fitur baru berhasil dibuat.


In [3]:
def cap_outliers(df, columns):
    for col in columns:
        lower_limit = df[col].quantile(0.01)
        upper_limit = df[col].quantile(0.99)
        df[col] = np.clip(df[col], lower_limit, upper_limit)
    return df

cols_to_cap = ['monthly_income_usd', 'monthly_expenses_usd', 'savings_usd', 'loan_amount_usd']
df = cap_outliers(df, cols_to_cap)

print("Outliers telah ditangani menggunakan metode Capping.")

Outliers telah ditangani menggunakan metode Capping.


In [4]:
# 1. Ordinal Encoding untuk Education Level
education_map = {'High School': 1, 'Bachelor': 2, 'Master': 3, 'PhD': 4}
df['education_level_encoded'] = df['education_level'].map(education_map)

# 2. Label Encoding untuk Binary (Gender & has_loan)
le = LabelEncoder()
df['gender_encoded'] = le.fit_transform(df['gender'])
df['has_loan_encoded'] = le.fit_transform(df['has_loan'])

# 3. One-Hot Encoding untuk Region
df = pd.get_dummies(df, columns=['region', 'age_group'], prefix=['region', 'age'])

print("Encoding selesai.")

Encoding selesai.


In [5]:
# Memilih fitur numerik final untuk pemodelan
final_features = [
    'age', 'monthly_income_usd', 'monthly_expenses_usd', 'savings_usd', 
    'loan_amount_usd', 'loan_term_months', 'monthly_emi_usd', 'loan_interest_rate_pct',
    'debt_to_income_ratio', 'credit_score', 'savings_to_income_ratio',
    'disposable_income', 'expense_to_income_ratio', 'loan_to_income_ratio',
    'education_level_encoded', 'gender_encoded', 'has_loan_encoded'
]

# Menambahkan kolom hasil One-Hot Encoding (Region & Age Groups)
final_features += [col for col in df.columns if 'region_' in col or 'age_' in col]

# Scaling
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[final_features] = scaler.fit_transform(df[final_features])

display(df_scaled[final_features].head())

,age,monthly_income_usd,monthly_expenses_usd,savings_usd,loan_amount_usd,loan_term_months,monthly_emi_usd,loan_interest_rate_pct,debt_to_income_ratio,credit_score,...,has_loan_encoded,region_Africa,region_Asia,region_Europe,region_North America,region_Other,age_Gen Z,age_Millennials,age_Gen X,age_Boomers
0,0.840174,-0.257847,-0.900767,0.661689,-0.656628,-0.557867,-0.477399,-0.698454,-0.317626,-0.913468,...,-0.817830,-0.494268,-0.503188,-0.501648,-0.504632,2.015122,-0.392274,-0.495765,-0.635706,1.273173
1,-1.630114,-0.257826,-0.033004,0.095383,0.303681,-0.214564,0.287167,0.708730,0.054525,-0.202869,...,1.222748,-0.494268,-0.503188,-0.501648,1.981643,-0.496248,2.549238,-0.495765,-0.635706,-0.785439
2,-1.563349,-0.643479,-1.031568,-0.063436,-0.656628,-0.557867,-0.477399,-0.698454,-0.317626,1.124001,...,-0.817830,2.023193,-0.503188,-0.501648,-0.504632,-0.496248,2.549238,-0.495765,-0.635706,-0.785439
3,-1.229526,0.986785,1.483884,0.328440,-0.044685,-0.328998,0.283010,1.827721,-0.094335,-0.718525,...,1.222748,-0.494268,-0.503188,1.993431,-0.504632,-0.496248,-0.392274,2.017084,-0.635706,-0.785439
4,0.639880,0.583378,1.262434,1.159417,-0.656628,-0.557867,-0.477399,-0.698454,-0.317626,-0.372658,...,-0.817830,2.023193,-0.503188,-0.501648,-0.504632,-0.496248,-0.392274,-0.495765,-0.635706,1.273173


In [6]:
output_path = "../data/processed/personal_finance_final_ml.csv"
df_scaled.to_csv(output_path, index=False)
print(f"✅ Tahap 3 Selesai! Data siap untuk pemodelan disimpan di: {output_path}")

✅ Tahap 3 Selesai! Data siap untuk pemodelan disimpan di: ../data/processed/personal_finance_final_ml.csv


In [ ]:
Summary of Insights & Strategy

Key Engineering Insights
Predictive Power of New Features: Berdasarkan analisis korelasi, fitur baru disposable_income dan loan_to_income_ratio menunjukkan hubungan yang lebih kuat terhadap credit_score dibandingkan variabel mentah seperti monthly_income_usd saja.

Multicollinearity Mitigation: Melalui pengecekan VIF (Variance Inflation Factor), kita berhasil mengidentifikasi dan menangani fitur yang saling tumpang tindih (redundant). Hal ini sangat krusial agar model Logistic Regression atau Linear-based models di tahap selanjutnya tidak mengalami bias atau overfitting.

Robustness against Outliers: Penerapan teknik Capping pada persentil 1% dan 99% memastikan bahwa data ekstrem (seperti pendapatan yang sangat tinggi secara tidak wajar) tidak menarik rata-rata distribusi, sehingga model tetap stabil namun tidak kehilangan informasi penting.

Data Readiness: Proses StandardScaling telah menyamakan "lapangan bermain" bagi semua variabel. Fitur dengan satuan ribuan dolar kini memiliki skala yang sama dengan fitur rasio (0-1), yang merupakan syarat mutlak bagi algoritma berbasis jarak seperti K-Means Clustering.

Business & Modeling Strategy
Risk-Adjusted Lending: Strategi pemodelan selanjutnya akan mengutamakan fitur loan_to_income_ratio. Nasabah dengan pendapatan tinggi tetap akan diprioritaskan sebagai "High Risk" jika rasio cicilan mereka terhadap pendapatan bersih (disposable income) sudah melewati ambang batas kritis.

Cluster-Ready Dataset: Dataset final personal_finance_final_ml.csv telah di-optimasi untuk tahap Segmentasi. Variabel kategori yang telah di-encode (seperti region dan education_level) kini memungkinkan algoritma untuk mengelompokkan nasabah bukan hanya berdasarkan angka, tapi juga konteks demografi.

Model Efficiency: Dengan eliminasi fitur yang memiliki VIF tinggi, kita telah mengurangi beban komputasi model tanpa mengurangi akurasi informasi, yang akan mempercepat proses training pada tahap Predictive Modeling.

Next Step: Tahap 4 - Clustering & Segmentation
Kita akan menggunakan dataset yang sudah diproses ini untuk melakukan Unsupervised Learning. Tujuannya adalah menemukan kelompok-kelompok tersembunyi (hidden segments) dari 32,424 record ini, guna mengidentifikasi siapa saja nasabah "Sultan yang Hemat", "Nasabah Terlilit Hutang", hingga "Nasabah Ideal".